In [1]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method


"""

'\nPurpose: To decompose the multi-somas for splitting\nusing the new decomposition method\n\n\n'

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-06-02 16:25:39,431 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-02 16:25:39,433 - settings - Setting database.user to celiib
INFO - 2021-06-02 16:25:39,433 - settings - Setting database.password to newceliipass
INFO - 2021-06-02 16:25:39,435 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-06-02 16:25:39,436 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-02 16:25:39,448 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-06-02 16:25:40,069 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-02 16:25:40,076 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-02 16:25:40,077 - settings - Setting database.user to celiib
INFO - 2021-06-02 16:25:40,086 - settings - Setting database.password to newceliipass
INFO - 2021-06-02 16:25:40,087 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-02 16:25:40,088 - settings - Setting database.user to celiib
INFO - 2021-06-02 16:25:40,089 - settings - Setting database.password to newceliipass
INFO - 2021-06-02 16:25:40,091 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-02 16:25:40,755 - settings - Setting enable_python_native_blobs to True


In [3]:
#so that it will have the adapter defined
from datajoint_utils import *

In [4]:
test_mode = True

# Debugging the contains method

In [5]:
import system_utils as su

In [6]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-06-02 16:25:40,860 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-02 16:25:40,861 - settings - Setting database.user to celiib
INFO - 2021-06-02 16:25:40,863 - settings - Setting database.password to newceliipass
INFO - 2021-06-02 16:25:40,865 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-02 16:25:40,865 - settings - Setting database.user to celiib
INFO - 2021-06-02 16:25:40,866 - settings - Setting database.password to newceliipass
INFO - 2021-06-02 16:25:40,870 - settings - Setting enable_python_native_blobs to True


Sleeping 119 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-06-02 16:25:41,483 - settings - Setting enable_python_native_blobs to True


In [7]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

import classification_utils as clu
import proofreading_utils as pru

import datajoint as dj

from pykdtree.kdtree import KDTree
import trimesh_utils as tu
import proofreading_utils as pru
import numpy as np

# Defining the Table

In [8]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [9]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 7827


In [10]:
# minnie,schema = du.configure_minnie_vm()
# minnie.AutoProofreadNeurons5.drop()
# minnie.AutoProofreadStats5.drop()
# minnie.AutoProofreadSynapse5.drop()
# minnie.AutoProofreadSynapseErrors5.drop()
# minnie.schema.external['faces'].delete(delete_external_files=True)
# minnie.schema.external['skeleton'].delete(delete_external_files=True)

# Proofreading Version

In [11]:
@schema
class AutoProofreadVersion(dj.Manual):
    definition="""
    proof_version      : tinyint unsigned                   # key by which to lookup the decomposition process version
    ---
    description          : varchar(256)                 # new parts of the iteration of the decomposition process
    """
versions=[[0,"exc,inh rules"],
         [1,"eliminated presyns on dendrite"],
         [2,"high fidelity axons"],
         [3,"improved crossover"],
         [4,"better crossover and more axon rules"],
         [5,"fixed stitching on axon"]]


dict_to_write = [dict(proof_version=k,description=v) for k,v in versions]
AutoProofreadVersion.insert(dict_to_write,skip_duplicates=True)
AutoProofreadVersion()

proof_version key by which to lookup the decomposition process version,description new parts of the iteration of the decomposition process
0,"exc,inh rules"
1,eliminated presyns on dendrite
2,high fidelity axons
3,improved crossover
4,better crossover and more axon rules
5,fixed stitching on axon


# Defining the Synapse Table

In [12]:
@schema
class AutoProofreadSynapseAllen5(dj.Manual):
    definition="""
    synapse_id           : bigint unsigned              # synapse index within the segmentation
    synapse_type: enum('presyn','postsyn')
    ver                  : decimal(6,2)                 # the version number of the materializaiton
    ---
    segment_id           : bigint unsigned              # segment_id of the cell. Equivalent to Allen 'pt_root_id
    split_index          : tinyint unsigned             # the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
    nucleus_id           : int unsigned                 # id of nucleus from the flat segmentation  Equivalent to Allen: 'id'. 
    skeletal_distance_to_soma=NULL : double #the length (in um) of skeleton distance from synapse to soma (-1 if on the soma)
    """

@schema
class AutoProofreadSynapseErrorsAllen5(dj.Manual):
    definition="""
    synapse_id           : bigint unsigned              # synapse index within the segmentation
    synapse_type: enum('presyn','postsyn')
    ver                  : decimal(6,2)                 # the version number of the materializaiton
    ---
    segment_id           : bigint unsigned              # segment_id of the cell. Equivalent to Allen 'pt_root_id
    split_index          : tinyint unsigned             # the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
    nucleus_id           : int unsigned                 # id of nucleus from the flat segmentation  Equivalent to Allen: 'id'. 
    skeletal_distance_to_soma=NULL : double #the length (in um) of skeleton distance from synapse to soma (-1 if on the soma)
    """

# Defining the Proofreading Stats Table

In [13]:
"""
This table will include the following information:

1) Filtering Info
2) Synapse Stats for Individual Neuron
3) Synapse Stats for Segment


**** thing need to add:
1) Axon faces
2) Axon length/area
2) Neuron faces
3) n_presyn_error_syn_non_axon
"""

'\nThis table will include the following information:\n\n1) Filtering Info\n2) Synapse Stats for Individual Neuron\n3) Synapse Stats for Segment\n\n\n**** thing need to add:\n1) Axon faces\n2) Axon length/area\n2) Neuron faces\n3) n_presyn_error_syn_non_axon\n'

In [14]:
@schema
class AutoProofreadStatsAllen5(dj.Manual):
    definition="""
    -> minnie.Decomposition()
    split_index          : tinyint unsigned             # the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
    -> minnie.AutoProofreadVersion()   # the version of code used for this cell typing classification
    ---
    mesh_faces: <faces>                      # faces indices that were saved off as belonging to proofread neuron (external storage)
    axon_faces: <faces>                      # faces indices that were saved off as belonging to proofread neuron's axon (external storage)
    
    axon_skeleton: <skeleton>      # the skeleton of the axon of the final proofread neuorn
    dendrite_skeleton: <skeleton>  # the skeleton of the dendrite branches of the final proofread neuorn
    neuron_skeleton: <skeleton>    # the skeleton of the entire neuron
    
    axon_on_dendrite_merges_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    axon_on_dendrite_merges_error_length=NULL : double #the length (in um) of skeleton distance canceled out by filter
    
    low_branch_clusters_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    low_branch_clusters_error_length=NULL : double #the length (in um) of skeleton distance canceled out by filter
    
    dendrite_on_axon_merges_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    dendrite_on_axon_merges_error_length =NULL: double #the length (in um) of skeleton distance canceled out by filter
    
    
    crossovers_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    crossovers_error_length=NULL : double #the length (in um) of skeleton distance canceled out by filter
    
    high_degree_coordinates_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    high_degree_coordinates_error_length=NULL : double #the length (in um) of skeleton distance canceled out by filter
    
    
    
    # ---------- new filters for v4 Stats ------------
    high_degree_branching_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    high_degree_branching_error_length=NULL : double #the length (in um) of skeleton distance canceled out by filter
    
    axon_webbing_t_merges_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    axon_webbing_t_merges_error_length=NULL : double #the length (in um) of skeleton distance canceled out by filter
    
    thick_t_merge_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    thick_t_merge_error_length=NULL : double #the length (in um) of skeleton distance canceled out by filter
    
    double_back_and_width_change_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    double_back_and_width_change_error_length=NULL : double #the length (in um) of skeleton distance canceled out by filter
    
    axon_fork_divergence_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    axon_fork_divergence_error_length=NULL : double #the length (in um) of skeleton distance canceled out by filter
    
    # ---------- new filers for v5 Stats ----------
    
    width_jump_up_dendrite_error_area=NULL :double #the area (in um ^ 2) of the faces canceled out by filter FOR DENDRITES
    width_jump_up_dendrite_error_length =NULL: double #the length (in um) of skeleton distance canceled out by filter FOR DENDRITES
    
    width_jump_up_axon_error_area=NULL :double # the area (in um ^ 2) of the faces canceled out by filter FOR AXONS
    width_jump_up_axon_error_length =NULL: double #the length (in um) of skeleton distance canceled out by filter FOR AXONS
    
    double_back_dendrite_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter FOR DENDRITES
    double_back_dendrite_error_length =NULL: double #the length (in um) of skeleton distance canceled out by filter FOR DENDRITES
    
    double_back_axon_thin_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter 
    double_back_axon_thin_error_length =NULL: double #the length (in um) of skeleton distance canceled out by filter
    
    double_back_axon_thick_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter 
    double_back_axon_thick_error_length =NULL: double #the length (in um) of skeleton distance canceled out by filter
    
    # ------------ For local valid synapses to that split_index
    n_valid_syn_presyn_for_split: int unsigned
    n_valid_syn_postsyn_for_split : int unsigned
    n_presyn_error_syn_non_axon :int unsigned
    
    # ------------ For global stats belonging to the whole segment
    # For the whole segment
    n_presyn_error_syn: int unsigned
    n_postsyn_error_syn: int unsigned
    total_error_synapses: int unsigned
    
    total_presyns: int unsigned 
    total_postsyns: int unsigned 
    total_synapses:int unsigned
    
    perc_error_presyn=NULL: double
    perc_error_postsyn=NULL: double
    
    overall_percent_error=NULL: double
    
    limb_branch_to_cancel: longblob # stores the limb information from 
    red_blue_suggestions: longblob
    """
    
    

# Creating the Auto Proofread Neuron Table

In [15]:
import numpy as np
import time
import classification_utils as clu

axon_version = 5
proof_version = 5

verbose = True

@schema
class AutoProofreadNeuronsAllen5(dj.Computed):
    definition="""
    -> minnie.Decomposition()
    split_index          : tinyint unsigned             # the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
    -> minnie.AutoProofreadVersion()             # the version of code used for this cell typing classification
    -> minnie.DecompositonAxonVersion()           # the version of the axon processing
    ---
    multiplicity  : tinyint unsigned   # the total number of neurons that came from the parent segment id
    # -------- Important Excitatory Inhibitory Classfication ------- #
    cell_type_predicted: enum('excitatory','inhibitory','other','unknown') # morphology predicted by classifier
    spine_category: enum('no_spined','sparsely_spined','densely_spined')
    
    n_axons: tinyint unsigned             # Number of axon candidates identified
    n_apicals: tinyint unsigned             # Number of apicals identified
    
    axon_length: double  # length (in um) of the classified axon skeleton
    axon_area: double # # area (in um^2) of the classified axon
    
    # ----- Soma Information ----#
    nucleus_id           : int unsigned                 # id of nucleus from the flat segmentation  Equivalent to Allen: 'id'.
    nuclei_distance      : double                    # the distance to the closest nuclei (even if no matching nuclei found)
    n_nuclei_in_radius   : tinyint unsigned          # the number of nuclei within the search radius of 15000 belonging to that segment
    n_nuclei_in_bbox     : tinyint unsigned          # the number of nuclei within the bounding box of that soma
    
    soma_x            : int unsigned                 # x coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)
    soma_y            : int unsigned                 # y coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)
    soma_z            : int unsigned                 # z coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)
    
    max_soma_n_faces     : int unsigned                 # The largest number of faces of the somas
    max_soma_volume      : int unsigned                 # The largest volume of the somas the (volume in billions (10*9 nm^3))
    
    # ---- Stores Neuron Mesh Faces (moved to AutoProofreadStats) --------
    
    
    # ------------- The Regular Neuron Information ----------------- #
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_not_processed_soma_containing_meshes : int unsigned  #the number of meshes with somas that were not processed
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint
    n_boutons: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    
    
    
    # ------ Information Used For Excitatory Inhibitory Classification -------- 
    axon_angle_maximum=NULL:double #the anlge of an identified axon
    spine_density_classifier:double              # the number of spines divided by skeletal length for branches analyzed in classification
    n_branches_processed: int unsigned                 # the number branches used for the spine density analysis
    skeletal_length_processed: double                 # The total skeletal length of the viable branches used for the spine density analysis
    n_branches_in_search_radius: int unsigned                 # the number branches existing in the search radius used for spine density
    skeletal_length_in_search_radius : double         # The total skeletal length of the branches existing in the search radius used for spine density
    
    

    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """
    
    #key_source = minnie.Decomposition() & minnie.NucleiSegmentsRun2()
    #key_source = (minnie.Decomposition() & minnie.NucleiSegmentsRun2() 
    #              & minnie.DecompositionAxon().proj()) #& dict(segment_id=864691136361533410)
    key_source = (minnie.Decomposition() & minnie.NucleiSegmentsRun4() 
                  & minnie.DecompositionAxon().proj() 
              & (minnie.AutoProofreadNeurons4() & "spine_category = 'densely_spined'").proj()
             ) 
    

    def make(self,key):
        """
        Pseudocode:
        1) Pull Down All of the Neurons
        2) Get the nucleus centers and the original mesh

        """
        
        # 1) Pull Down All of the Neurons
        segment_id = key["segment_id"]
        
        whole_pass_time = time.time()
        

        curr_output = pru.proofreading_table_processing(key,
                                  proof_version=proof_version,
                                  axon_version  = axon_version,
                                  compute_synapse_to_soma_skeletal_distance=True,
                                  perform_axon_classification = False,
                                  high_fidelity_axon_on_excitatory = False,
                                 verbose=True,)    
        # ------ Writing the Data To the Tables ----- #
            
            
        AutoProofreadSynapse_keys = curr_output["AutoProofreadSynapse_keys"]
        AutoProofreadSynapseErrors_keys = curr_output["AutoProofreadSynapseErrors_keys"]
        AutoProofreadNeurons_keys = curr_output["AutoProofreadNeurons_keys"]
        filtering_info_list = curr_output["filtering_info_list"]
        synapse_stats_list = curr_output["synapse_stats_list"]
        total_error_synapse_ids_list = curr_output["total_error_synapse_ids_list"]
        neuron_mesh_list = curr_output["neuron_mesh_list"]
        axon_mesh_list = curr_output["axon_mesh_list"]
        neuron_split_idxs = curr_output["neuron_split_idxs"]
        
        axon_skeleton_list = curr_output["axon_skeleton_list"]
        dendrite_skeleton_list = curr_output["dendrite_skeleton_list"]
        neuron_skeleton_list = curr_output["neuron_skeleton_list"]
            
        
        # Once have inserted all the new neurons need to compute the stats
        if verbose:
            print("Computing the overall stats")
            
        overall_syn_error_rates = pru.calculate_error_rate(total_error_synapse_ids_list,
                        synapse_stats_list,
                        verbose=True)
        
        
        # Final Part: Create the stats table entries and insert
        
        proofread_stats_entries = []
        
        stats_to_make_sure_in_proofread_stats = [
            
         'axon_on_dendrite_merges_error_area',
         'axon_on_dendrite_merges_error_length',
         'low_branch_clusters_error_area',
         'low_branch_clusters_error_length',
         'dendrite_on_axon_merges_error_area',
         'dendrite_on_axon_merges_error_length',
         'double_back_and_width_change_error_area',
         'double_back_and_width_change_error_length',
         'crossovers_error_area',
         'crossovers_error_length',
         'high_degree_coordinates_error_area',
         'high_degree_coordinates_error_length',
        ]
        
        
        for sp_idx,split_index in enumerate(neuron_split_idxs):
            
            #write the AutoProofreadNeurons and AutoProofreadSynapse Tabel
            keys_to_write = AutoProofreadSynapse_keys[sp_idx]
            AutoProofreadSynapseAllen5.insert(keys_to_write,skip_duplicates=True)
            
            keys_to_write_errors = AutoProofreadSynapseErrors_keys[sp_idx]
            AutoProofreadSynapseErrorsAllen5.insert(keys_to_write_errors,skip_duplicates=True)
            
            
            
            new_key = AutoProofreadNeurons_keys[sp_idx]
            self.insert1(new_key,skip_duplicates=True,allow_direct_insert=True)
            
            synapse_stats = synapse_stats_list[sp_idx]
            filtering_info = filtering_info_list[sp_idx]
            limb_branch_to_cancel = pru.extract_from_filter_info(filtering_info,
                            name_to_extract="limb_branch_dict_to_cancel")
                            
            
            red_blue_suggestions = pru.extract_from_filter_info(filtering_info,
                            name_to_extract = "red_blue_suggestions")
            
            curr_key = dict(key,
                           split_index = split_index,
                           proof_version = proof_version,
                           
                             mesh_faces = neuron_mesh_list[sp_idx],
                            axon_faces = axon_mesh_list[sp_idx],
                            
                            axon_skeleton = axon_skeleton_list[sp_idx],
                            dendrite_skeleton = dendrite_skeleton_list[sp_idx],
                            neuron_skeleton = neuron_skeleton_list[sp_idx],
                         

                            # ------------ For local valid synapses to that split_index
                            n_valid_syn_presyn_for_split=synapse_stats["n_valid_syn_presyn"],
                            n_valid_syn_postsyn_for_split=synapse_stats["n_valid_syn_postsyn"],
                            n_presyn_error_syn_non_axon=synapse_stats["n_errored_syn_presyn_non_axon"],
                            
                            limb_branch_to_cancel = limb_branch_to_cancel,
                            red_blue_suggestions = red_blue_suggestions,
                           
                           
                           )
            
            
            for s in stats_to_make_sure_in_proofread_stats:
                if s not in filtering_info.keys():
                    curr_key[s] = None
            
            filter_key = {k:np.round(v,2) for k,v in filtering_info.items() if "area" in k or "length" in k}
            curr_key.update(filter_key)
            curr_key.update(overall_syn_error_rates)
            
            proofread_stats_entries.append(curr_key)
            
        
        AutoProofreadStatsAllen5.insert(proofread_stats_entries,skip_duplicates=True)
            
#         for pse in proofread_stats_entries:
#             AutoProofreadStats4.insert1(pse,skip_duplicates=True)
            

        print(f"\n\n ***------ Total time for {key['segment_id']} = {time.time() - whole_pass_time} ------ ***")


# Running the Populate

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__auto_proofread_neurons_allen5'")
(curr_table)#.delete()# & "status='error'")
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()


In [16]:
import time
pru = reload(pru)
nru = reload(nru)
import neuron_searching as ns
ns = reload(ns)
clu = reload(clu)
du = reload(du)
import random

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    AutoProofreadNeuronsAllen5.populate(reserve_jobs=True, suppress_errors=True, order="random")
else:
    AutoProofreadNeuronsAllen5.populate(reserve_jobs=True, suppress_errors=False, order="random")
print('Populate Done')

print(f"Total time for AutoProofreadNeuron5 populate = {time.time() - start_time}")

INFO - 2021-06-02 16:26:03,589 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-02 16:26:03,591 - settings - Setting database.user to celiib
INFO - 2021-06-02 16:26:03,592 - settings - Setting database.password to newceliipass
INFO - 2021-06-02 16:26:03,593 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-02 16:26:03,594 - settings - Setting database.user to celiib
INFO - 2021-06-02 16:26:03,594 - settings - Setting database.password to newceliipass
INFO - 2021-06-02 16:26:03,601 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-02 16:26:04,373 - settings - Setting enable_python_native_blobs to True


Populate Started


INFO - 2021-06-02 16:26:06,728 - autopopulate - Found 48313 keys to populate
INFO - 2021-06-02 16:26:06,759 - connection - Transaction started
INFO - 2021-06-02 16:26:06,761 - autopopulate - Populating: {'segment_id': 864691135207738489, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'ver': Decimal('30.00'), 'process_version': 0, 'index': 0}




------- AutoProofreadNeuron 864691135207738489  ----------
**Using table __decomposition_axon for table_to_neuron_objs**
Decompressing Neuron in minimal output mode...please wait
Dont need spine recalculation
Number of Neurons found = 1
Number of Neurons found =1
Number of Corresponding Nuclei = 1
nucleus_ids = [565848]
nucleus_centers = [[1255232  947520  796920]]

-----Working on Neuron Split 0-----

   --> Part A: Proofreading the Neuron ----
---- Part A: NOT Attempting to split neuron --------

--- Working on Neuron 0 ---


 ------ Part B: Axon Classification ---- 


Skipping Axon Classification
endnodes_to_test = [1274 1506 1990 2037]
Skeleton paths distances = [24955.71210678  5826.69225949  6192.86091871  7811.80190811]
Filtered indexes = [0]
len(filtered_skeletons) = 1
sk_angles = [141.6905637]
local_axon_angles = [141.6905637]


 ------ Part C: Inhibitory Excitatory Classification ---- 


spine_category = densely_spined
Inhibitory Excitatory Classification = 9.81160163879394

----------------------- FINISHED Running Filter ------------------



 --Filter high_degree_branching Results --

--- Working on filter 2:
function = axon_webbing_t_merges
function __name__ = filter_away_webbing_t_merges
function arguments = {'child_width_maximum': 75, 'parent_width_maximum': 75, 'axon_only': True, 'error_if_web_is_none': True, 'web_size_threshold': 120, 'web_size_type': 'ray_trace_median', 'web_above_threshold': True}


----------------------- Running Filter ------------------
two_downstream_thin_axon_limb_branch = {}
wide_angled_children= {}
Final web t error limb branch dict = {}
limb_branch_dict_to_cancel = {}
limb_branch_dict was empty so returning original neuron
----------------------- FINISHED Running Filter ------------------



 --Filter axon_webbing_t_merges Results --

--- Working on filter 3:
function = thick_t_merge
function __name__ = filter_away_thick_t_merge
function arguments = {}


----------------------- Running Filter ------------------
two_downstr

...

Angle between 36 and 38 = 16.92 
Angle between 37 and 38 = 137.03 
Final Matches = [[36, 37], [36, 38]], Final Matches Angle = [27.33, 16.92]
matched_edges = [[36, 37], [36, 38]]matched_edges_angles = [27.33, 16.92]
Step 2: Edges with worst case scenario matching = 65
Remaining Edges = [(36, 37), (36, 38)], Remaining Nodes = [35, 36, 37, 38]
edges_to_remove_by_width = []
Step 2: Edges after widht mismatch
Remaining Edges = [(36, 37), (36, 38)], Remaining Nodes = [35, 36, 37, 38]
--Working on edge [36 37]--
--Working on edge [36 38]--
Step 4: Definite Edges
Remaining Edges = [(36, 37), (36, 38)], Remaining Nodes = [35, 36, 37, 38]
upstream_subgraph = [35]
Possible Connections = [], angles = []
Step 5: Removing worst case edges
Remaining Edges = [(36, 37), (36, 38)]
Not doing kiss check because upstream_matches = []
winning_downstream = None,error_downstream = [36 37 38] 
coordinate [882924.05753682 879135.14154304 789728.47383437] had error branches [36 37 38]--------


 ----- Working o



 ----- Working on coordinate 15: [ 941001.04080576 1063057.5478652   872551.38067851]--------
For test node 197, sibling nodes were: [202 206]
overlap = [202 206]
With test node equal to the downstream node
upstream = 207, downstream_nodes = [202 206 197]
branches_at_coord = [197 202 206 207]
widths_in_branches = [113.37756361  69.81485522 102.01323695 150.03123407]
coordinate_branches = [197, 202, 206, 207]
197 = red
202 = aqua
206 = purple
207 = green
Angle between 197 and 202 = 19.2 
Angle between 197 and 206 = 157.4 
Angle between 197 and 207 = 75.45 
Angle between 202 and 206 = 21.94 
Angle between 202 and 207 = 121.31 
Angle between 206 and 207 = 77.1 
Final Matches = [[197, 202], [202, 206]], Final Matches Angle = [19.2, 21.94]
matched_edges = [[197, 202], [202, 206]]matched_edges_angles = [19.2, 21.94]
Step 2: Edges with worst case scenario matching = 65
Remaining Edges = [(197, 202), (202, 206)], Remaining Nodes = [197, 202, 206, 207]
edges_to_remove_by_width = []
Step 2: Ed

Angle between 299 and 321 = 152.32 
Angle between 299 and 322 = 125.99 
Angle between 299 and 323 = 116.89 
Angle between 299 and 324 = 35.94 
Angle between 319 and 320 = 168.9 
Angle between 319 and 321 = 165.79 
Angle between 319 and 322 = 144.9 
Angle between 319 and 323 = 118.31 
Angle between 319 and 324 = 17.64 
Angle between 320 and 321 = 168.42 
Angle between 320 and 322 = 155.77 
Angle between 320 and 323 = 110.9 
Angle between 320 and 324 = 21.87 
Angle between 321 and 322 = 153.67 
Angle between 321 and 323 = 104.11 
Angle between 321 and 324 = 30.76 
Angle between 322 and 323 = 90.06 
Angle between 322 and 324 = 43.23 
Angle between 323 and 324 = 47.46 
Final Matches = [[299, 324], [319, 324], [320, 324], [321, 324], [322, 324], [323, 324]], Final Matches Angle = [35.94, 17.64, 21.87, 30.76, 43.23, 47.46]
matched_edges = [[299, 324], [319, 324], [320, 324], [321, 324], [322, 324], [323, 324]]matched_edges_angles = [35.94, 17.64, 21.87, 30.76, 43.23, 47.46]
Step 2: Edges wit

After skeletal restrictions, limb_branch_dict_restriction = {'L4': array([ 3,  7, 11, 14, 20, 25, 42, 51])}
limb_branch_dict_to_cancel = {}
limb_branch_dict was empty so returning original neuron
----------------------- FINISHED Running Filter ------------------



 --Filter double_back_axon_thin Results --

--- Working on filter 8:
function = double_back_axon_thick
function __name__ = filter_away_double_back_axon_thick
function arguments = {}


----------------------- Running Filter ------------------
After skeletal restrictions, limb_branch_dict_restriction = {'L4': array([ 13,  56,  61, 107])}
limb_branch_dict_to_cancel = {'L4': array([107])}
After disconnecte effect, removed_limb_branch_dict = {'L4': [95, 106, 107]}
total_sk_distance = 10.859683976209151, total_area = 19.754164261784997
----------------------- FINISHED Running Filter ------------------



 --Filter double_back_axon_thick Results --

--- Working on filter 9:
function = axon_fork_divergence
function __name__ = filter

TypeError: object of type 'NoneType' has no len()